In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier

ModuleNotFoundError: No module named 'tslearn'

In [ ]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
#import models with pickle
import pickle
clf1 = pickle.load(open('models/catboost_model.sav', 'rb'))
clf2 = pickle.load(open('models/extra_model.sav', 'rb'))
clf3 = pickle.load(open('models/svm_model.sav', 'rb'))
clf4 = pickle.load(open('models/xgboost_model.sav', 'rb'))


In [ ]:
#voting classifier
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4)], voting='hard')


In [ ]:
#commento perché da errore con catboost :(
'''
scores = cross_val_score(eclf1, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)
print(scores.mean())
print(scores.std())
'''

#0.6773494776392961 / 0.00782106085974223

In [ ]:
eclf2 = VotingClassifier(estimators=[ ('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4)], voting='soft')
scores = cross_val_score(eclf2, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)

In [ ]:
print(scores.mean())
print(scores.std())
#0.6990398567326491 / 0.01341123010967938 MinMaxScaler
#0.7064684139784948 / 0.010452913689302797 StandardScaler